In [6]:
import tensorflow as tf
import tensorflow.keras as keras
import cv2
from src.lab1.lab1_challenge2 import *
from src.lab1.info_image import *
import numpy as np
from src.lab3.

In [7]:
def extract_mask(matrix, center, w, h):
    batch = [[0 for u in range(w)] for i in range(h)]
    w2 = w//2
    h2 = h//2
    for i in range(w2):
        for j in range(h2):
            batch[i][j] = matrix[center[0] - w2 +i][center[1] - h2 +j]
            batch[w2+i][h2+j] = matrix[center[0] + i][center[1] + j] 
    
    return batch

def extract_batch(matrix, center, w, h):
    batch = [[[0 for u in range(3)] for i in range(w)] for j in range(h)]
    batch = np.ndarray(shape=(w,h,3))
    w2 = w//2
    h2 = h//2
    for i in range(w2):
        for j in range(h2):
            for k in range(3):
                batch[i][j][k] = matrix[center[0] - w2 + i+1][center[1] - h2 + j+1][k]
                batch[w2+i][h2+j][k] = matrix[center[0] + i+1][center[1] + j+1][k]
                batch[i][h2+j][k] = matrix[center[0] -w2 + i+1][center[1] + j+1][k]
                batch[w2+i][j][k] = matrix[center[0] + i+1][center[1] -h2+ j+1][k]
    return batch

In [17]:
data = get_all_masks(2)
X_train, y_train = [], []
# On crée les jeux d'entrainements....
for img_name, mask in data:
    img = cv2.imread(img_name)
    for (roi, center, w, h, angle) in sliding_windows(mask, 32, 32):
        X_train.append(extract_batch(img, center, 32, 32))
        y_train.append(extract_mask(mask, center, 32, 32))
        break

### On tente d'entrainer un truc random

In [18]:
# On crée le modèle
modele = keras.Sequential()

# On ajoute une couche de convolution
modele.add(keras.layers.Convolution2D(22, 3,3, input_shape=(32,32,3), activation = 'relu'))

#on ajoute une couche de pooling
modele.add(keras.layers.MaxPooling2D())

#on ajoute une couche dense
modele.add(keras.layers.Dense(32))

modele.compile(optimizer=tf.train.AdamOptimizer(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

modele.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 10, 10, 22)        616       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 22)          0         
_________________________________________________________________
dense_2 (Dense)              (None, 5, 5, 32)          736       
Total params: 1,352
Trainable params: 1,352
Non-trainable params: 0
_________________________________________________________________


In [19]:
history = modele.fit(np.array(X_train),
                    np.array(y_train))


ValueError: Error when checking target: expected dense_2 to have 4 dimensions, but got array with shape (3, 32, 32)